In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import hashlib
import os.path
import random
import re
import struct
import sys
import tarfile

import numpy as np
from six.moves import urllib
import tensorflow as tf

from tensorflow.python.framework import graph_util
from tensorflow.python.framework import tensor_shape
from tensorflow.python.platform import gfile
from tensorflow.python.util import compat

In [ ]:
FLAGS=None

#모든 파라미터들은 특정한 모델 아키텍쳐와 묶여이싿.
#우선 인셉션 v3를 사용할 것이다. 이는 텐서 이름이나 사이즈들을 포함하고 있다. 
# 만약 당신이 이 스크립트를 다른모델에 사용하고 싶다면 당신이 사용하는 네트워크에 반영하도록 텐서이름이나
# 사이즈들을 변경하면 된다.

DATA_URL="http://download.tensorflow.org/models/image/imagenet/inception-2015-12-05.tgz"
BOTTLENECK_TENSOR_NAME='pool_3/_reshape:0'
BOTTLENECK_TENSOR_SIZE=2048
MODEL_INPUT_WIDTH=299
MODEL_INPUT_HEIGHT=299
MODEL_INPUT_DEPTH=3
JPEG_DATA_TENSOR_NAME='DecodeJpeg/contents:0'
RESIZED_INPUT_TENSOR_NAME='ResizeBilinear:0'
MAX_NUM_IMAGES_PER_CLASS=2**27-1 #134M

In [12]:
def create_image_lists(image_dir,testing_percentage,validation_percentage):
    """file system으로부터 트레이닝 이미지를 가져와서 이미지들의 list를 만든다.
    이미지 디렉토리로부터 sub folder를 분석하고, 그들을 트레이닝, 테스팅, 검증 셋으로 나눈다. 
    그리고 각각의 라벨을 위한 이미지 리스트와 그들의 경로를 나타내는 자료구조로 반환한다.
    인수들(Args):
    image_dir:이미지들의 서브 폴더를 위한 entry를 포함한 dictionary a dictionary
    testing_percentage:전체이미지 중에 테스트에 사용되는 비율을 나타낸 int
    validataion_precentage: 전체이미지 중에 validataion을 위해 사용되는 비율을 나타내는 int
    
    반환값들(Returns):
    각각의 label subfolder를 위한 entry를 포함한 dictionary a dictionary
    (각각의 label에서 이미지들은 트레인, 테스트, 검증 셋으로 나누어져 있다.)
    """
    
    if not gifle.Exists(image_dir): # 이미지 파일이 존재하지 않을때 나타내는 문구
        print("Image directory ''"+image_dir+"'not found.'")
        return None
    
    result={}
    sub_dirs=[x[0] for x in gfile.walk(image_dir)]
    #root directory는 처음에 온다. 따라서 이를 스킵한다.
    is_root_dir=True
    for sub_dir in sub_dirs:
        if is_root_dir:
            is_root_dir=False
            continue
        extensions=['jpg','jpeg','JPG','JPEG']
        file_list=[]
        dir_name=os.path.basename(sub_dir)
        if dir_name==image_dir:
            continue
        print("Looking for images in '" + dir_name+"''")
        for extension in extensions:
            file_glob=os.path.join(image_dir,dir_name,'*.'+extension)
            file_list.extend(gfile.Glob(file_glob))
        if not file_list:
            print("No files founds")
            continue
        if len(file_list)<20:
            print("warning: folder has less than 20 images, which may cause issues.")
        elif len(file_list)>MAX_NUM_IMAGES_PER_CLASS:
            print("warning: folder { } has more than { } images. some images will never be selected".format(dir_name,MAX_NUM_IMAGES_PER_CLASS))
        
        label_name=re.sub(r'[^a-z0-9]+', ' ',dir_name.lower())
        training_images=[]
        testing_images=[]
        validation_images=[]
        
        for file_name in file_list:
            base_name=os.path.basename(file_name)
            #어떤 이미지로 리스트를 만들지 결정할 떄 파일 이름에 "_nohash_"가 포함되어 있으면 이를 무시할 수 있다.
            # 이를 이용해서 ㅇ데이터 셋을 만드는 사람은 서로 비슷한 사진들을 그룹핑할 수 있따. 
            # 예를 들어 plant disease를 데이터셋을 만들기 위해서 여러장의 잎사귀를 그룹핑할 수 있다.
            hash_name=re.sub(r'__nohash__.*$', '',file_name)
            #  이는 일종의 마법처럼 보일 수 있다. 하지만, 우리는 이 파일이 training sets로 갈지, testing sets로 갈지, validation sets로 갈지 결정해야만 한다.
            # 그리고 우리는 더많은 파일들이 추가되더라도, 같은 set에 이미 존재하는 파일들이 유지되길 원한다.
            # 그렇게 하기 위해서는, 우리는 파일 이름 그자체로부터 결정하는 안정적인 방법이 있어야만 한다.
            # 따라서, 우리는 파일 이름을 hash하고, 이를 이를 할당하는데 사용하는 확률을 결정하는데 사용한다.
            
            hash_name_hashed=hashlib.sha1(compat.as_bytes(hash_name)).hexdigest()
            percentage_hash=((int(hash_name_hashed,16)%
                             (MAX_NUM_IMAGES_PER_CLASS+1))*
                            (100.0/MAX_NUM_IMAGES_PER_CLASS))
            
            if percentage_hash < validation_percentage:
                validation_images.append(base_name)
            elif percentage_hash < (testing_percentage + validation_percentage):
                testing_images.append(base_name)
            else:
                training_images.append(base_name)
        
        result[label_name]={'dir':dir_name,
                           'training':training_images,
                           'testing':testing_images,
                           'validation':validation_images}
        
        return result
    

In [13]:
def get_image_path(iamge_lists,label_name,index,image_dir,category):
    """주어진 index에 대한 이미지 경로를 리턴한다.
    인수들(Args):
    image_lists: 각각의 라벨에 대한 트레이닝 이미지들의 사전
    label_name:우리가 얻고자하는 이미지의 label string
    index: 우리가 얻고자하는 이미지의 int offset.이는 레이블에 대한 가능한 이미지의 개수에 따라 모듈화될 것이다
    image_dir:training 이미지들의 subfolder를 포함하고 있는 root folder string
    category:트레이닝, 테스트, 검증셋으로 이미지에 pull할 name string
    반환값들(Returns):
    요청된 파라미터들이 만나게 될 이미지에 대한 파일 시스템 경로 string"""
    
    if label_name not in image_lists:
        tf.logging.fatal('Label does not in exist %s.', label_name)
    label_lists=image_lists[label_name]
    
    if category not in label_lists:
        tf.logging.fatal("Category does not exist %s.", category)
    category_list=label_lists[category]
    
    if not category_list:
        tf.logging.fatal('Label %s has no images in the category %s', label_name,category)
        
    mod_index=index % len(category_list)
    base_name=category_list[mode_index]
    sub_dir=label_lists['dir']
    full_path=os.path.join(image_dir,sub_dir,base_name)
    
    return full_path
    
    

In [ ]:
def get_bottleneck_path(image_lists,label_name,index,bottleneck_dir,category):
    """주어진 인덱스에 따라서 병목적으로 몰린 파일들의 경로들 보내준다.
    인수들(Args):
    image_lists:각 라벨에 해당하는 트레이닝 이미지의 사전
    label_name: 우리가 얻길 원하는 이미지 라벨 string
    indes: 위와동일
    bottleneck_dir: 병목된 값들의 캐쉬파일의 string
    category: 이미지의 풀 네임
    반환값들(Returns):
    요청된 파라미터들이 만나게 될 이미지에 대한 파일 시스템 경로"""
    
    return get_image_path(image_lists,label_name,index,bottleneck_dir,category)+'.txt'

def create_inception_graph():
    """저장된 그래프def로 부터 그래프를 만들고, 그래프 객체를 반환한다. 
    반환값들(Returns):
    훈련된 인셉션 네트워크를 가진 그래프, 우리가 조작하고 싶어하는 텐서들"""
    
    with tf.Graph().as_default() as graph:
        model_filename=os.path.join(FLAGS.model_dir,'classify_image_graph_def.pb')
        with gfile.FastGFile(model_filename,'rb') as f:
            graph_def=tf.GraphDef()
            graph_def.ParseFormString(f.read())
            bottleneck_tensor,jpeg_data_tensor,resized_input_tnesor=(
                tf.import_graph_def(graph_def,name="",return_elements=[
                    BOTTLENECK_TENSOR_NAME,JPEG_DATA_TENSOR_NAME,
                    RESIZED_INPUT_TENSOR_NAME
                ]))
            
    return graph, bottleneck_tensor, jpeg_data_tensor, resized_input_tnesor


def run_bottleneck_on_image(sess,image_data,image_data_tensor,bottleneck_tensor):
    """bottlneck 압축 레이어를 뽑기 위한 과정
    인수들(Args):
    sess:현재 활성화된 텐서플로우 세션
    image_data: raw 이미지의 string
    image_data_tensor: 그래프에서 input data layer
    bottleneck_tensor: 소프트 맥스 하기전 layer
    반환값들(Returns):
    bottleneck_values의 array"""
    bottleneck_values=sess.run(bottleneck_tensor,{image_data_tensor:image_data})
    return bottleneck_values
    

In [20]:

def maybe_download_and_extract():
    """ 모델을 다운받아 가져온다. 만약에 미리 훈련된 모델이 없다면 
    텐서플로우 웹사이트에서 다운로드해서 압축을 푼다."""
    
    dest_directory = FLAGS.model_dir
    if not os.path.exists(dest_directory):
      os.makedirs(dest_directory)
    filename = DATA_URL.split('/')[-1]
    filepath = os.path.join(dest_directory, filename)
    if not os.path.exists(filepath):

      def _progress(count, block_size, total_size):
          sys.stdout.write('\r>> Downloading %s %.1f%%' %
                       (filename,
                        float(count * block_size) / float(total_size) * 100.0))
          sys.stdout.flush()

    filepath, _ = urllib.request.urlretrieve(DATA_URL,
                                             filepath,
                                             _progress)
    print()
    statinfo = os.stat(filepath)
    print('Successfully downloaded', filename, statinfo.st_size, 'bytes.')
    tarfile.open(filepath, 'r:gz').extractall(dest_directory)


In [21]:
def ensure_dir_exits(dir_name):
    """파일이 디스크에 있는지 확인한다.
    인수들(Args):
    dir_name: 우리가 만들려 하는 폴더에 대한 경로 string"""
    if not os.path.exist(dir_name):
        os.makedirs(dir_name)
        
def write_list_of_floats_to_file(list_of_float,file_path):
    """주어진 float형식의 list를 binary 파일로 만든다."""
    
    s=struck.pack('d' * BOTTLENECK_TENSOR_SIZE,*list_of_floats)
    with open(file_path,'wb') as f:
        f.write(s)
        
def read_list_of_floats_from_file(file_path):
    """주어진 경로에서 float의 list들을 읽어온다"""
    
    with open(file_path,'rb') as f:
        s=struck.unpack('d'* BOTTLENECK_TENSOR_SIZE,f.read())
        return list(s)
    
bottleneck_path_2_bottleneck_value={}


In [24]:
def create_bottleneck_file(bottleneck_path,image_lists,label_name,index,
                          image_dir,category,sess,jpeg_data_tensor,bottleneck_tensor):
    """하나의 bottleneck file을 만든다."""
    print("creating bottleneck at "+ bottleneck_tensor)
    image_path=get_image_path(image_lists,label_name,index,image_dir,category)
    if not gfile.Exists(image_path):
        tf.logging.fatal("File does not exist %s ",image_path)
    image_data=gfile.FastGFile(image_path,'rb').read()
    
    try:
        bottleneck_values=run_bottleneck_on_image(sess,image_data,jpeg_data_tensor,bottleneck_tensor)
    except:
        raise RuntimeError('error during procession file %s' %image_path)
        
    bottleneck_string=','.join(str(x) for x in bottleneck_values)
    with open (bottleneck_string,'w') as bottleneck_file:
        bottleneck_file.write(bottleneck_string)
        

def get_or_create_bottleneck(sess,image_lists,label_name,index,image_dir,category,
                            bottleneck_dir,jpeg_data_tensor):
    """이미지에 대해서 bottleneck value를 반환하고 계산해준다.
    만약에 cached bottleneck data가 있으면 그것을 반환하고 그렇지 않으면 데이터를 계산하고 그것을 저장해서
    나중에 사용한다.
    인수들(Args):
    sess:현재 활성화된 텐서플로우 세션
    image_lists: 각 라벨에 해당하는 트레이닝 이미지들의 사전
    index:위와동일
    image_dir: 루트 폴더 string
    category: 트레인, 테스트, 검증에 쓸 이미지들의 string
    bottleneck_dir: bottleneck_value를 잡고 있는 cached 파일의 폴더명
    jpeg_data_tensor: 텐서 인풋데이터
    bottleneck_tensor: bottleneck_value들의 아웃풋 텐서
    반환값들(Returns):
    bottleneck layer에 의해서 만들어진 값들의 array
    """
    label_lists=image_lists[label_name]
    sub_dir=label_lists['dir']
    sub_dir_path=os.path.join(bottleneck_dir,sub_dir)
    ensure_dir_exists(sub_dir_path)
    bottleneck_path=get_bottleneck_path(image_lists,label_name,index,bottleneck_dir,category)
    if not os.path.exists(bottleneck_path):
        create_bottleneck_file(bottleneck_path,image_lists,label_name,index,image_dir,category,
                              sess,jpeg_data_tensor,bottleneck_tensor)
        
    with open(bottleneck_path,'r') as bottleneck_file:
        bottlenekc_string=bottleneck_file.read()
    
    did_hit_error=False
    try: 
        bottleneck_values=[float(x) for x in bottleneck_string.split(',')]
    
    except ValueError:
        print('invalid float around, recreating bottleneck')
        did_hit_error=True
    
    if did_hit_error:
        create_bottleneck_file(bottleneck_path,image_lists,label_name,index,image_dir,category,sess,
                              jpeg_data_tensor,bottleneck_tensor)
        with open(bottleneck_path,'r') as bottleneck_file:
            bottleneck_string=bottleneck_file.read()
        
        bottleneck_values=[float(x) for x in bottleneck_string.split(',')]
    
    return bottleneck_values   
    
    
    
def cache_bottlenecks(sess,image_lists,image_dir,bottleneck_dir,jpeg_data_tensor,bottleneck_tensor):
    """모든 트레이닝, 검증, 테스트 bottlenecks들이 cached인지 확인"""
    how_many_bottlenecks=0
    ensure_dir_exists(bottleneck_dir)
    for label_name,label_lists in image_lists.items():
        for category in ['training','testing','validation']:
            category_list=label_lists[category]
            for index,unused_base_name in enumerate(category_list):
                get_or_create_bottleneck(sess,image_lists,label_name,index,image_dir,category,
                                        bottleneck_dir,jpeg_data_tensor,bottleneck_tensor)
                
                how_many_bottlenecks+=1
                if how_many_bottlenecks % 100 ==0:
                    print(str(how_many_bottlenecks)+' bottleneck files created')
                

                


In [29]:
def get_random_cached_bottlenecks(sess,image_lists,how_many,category,bottleneck_dir,image_dir,
                                 jpeg_data_tensor,bottleneck_tensor):
    """cached 이미지들의 값을 검색하여 찾는다"""
    
    
    class_count=len(image_lists.keys())
    bottlenecks=[]
    ground_truths=[]
    filenames=[]
    if how_many >= 0:
        # bottlenecks의 랜덤 샘플값을 검색하여 찾는다.
        for unused_i in range(how_many):
            label_index=random_randrange(class_count)
            label_name=list(image_lists.keys())[label_index]
            image_index=random.randrange(MAX_NUM_IMAGES_PER_CLASS+1)
            image_name=get_image_path(image_lists,label_name,image_index,image_dir,category)
            bottleneck=get_or_create_bottleneck(sess,image_lists,label_name,image_index,
                                               image_dir,categoty,bottleneck_dir,jpeg_data_tensor,
                                               bottleneck_tensor)
            ground_truth=np.zeros(class_count,dtype=np.float32)
            ground_truth[label_index]=1.0
            bottlenecks.append(bottleneck)
            ground_truths.append(ground_truth)
            filenames.append(image_name)
            
    else:
        #모든 bottleneck을 가져온다.
        for label_index,label_name in enumerate(image_lists.keys()):
            for image_index,image_name in enumerate(image_lists[label_name][category]):
                image_name=get_image_path(image_lists,label_name,image_index,image_dir,category)
                bottleneck=get_or_create_bottleneck(sess,image_lists,label_name,image_index,
                                                   image_dir,category,bottleneck_dir,jpeg_data_tensor,
                                                   bottleneck_tensor)
                ground_truth=np.zeros(class_count,dtype=np.float32)
                ground_truth[label_index]=1.0
                bottlenecks.append(bottleneck)
                ground_truths.append(ground_truth)
                filenames.append(image_name)
                
    return bottlenecks, ground_truths, filenames




def get_random_distorted_bottlenecks(sess,image_lists,how_many,categoty,image_dir,input_jpeg_tensor,
                                    distorted_image,resized_input_tensor,bottleneck_tensor):
    """이미지를 왜곡한 이후에 트레인 이미지들의 bottleneck value를 가져온다."""
    
    class_count = len(image_lists.keys())
    bottlenecks = []
    ground_truths = []
    for unused_i in range(how_many):
        label_index = random.randrange(class_count)
        label_name = list(image_lists.keys())[label_index]
        image_index = random.randrange(MAX_NUM_IMAGES_PER_CLASS + 1)
        image_path = get_image_path(image_lists, label_name, image_index, image_dir,
                                    category)
        if not gfile.Exists(image_path):
          tf.logging.fatal('File does not exist %s', image_path)
        jpeg_data = gfile.FastGFile(image_path, 'rb').read()
        # Note that we materialize the distorted_image_data as a numpy array before
        # sending running inference on the image. This involves 2 memory copies and
        # might be optimized in other implementations.
        distorted_image_data = sess.run(distorted_image,
                                        {input_jpeg_tensor: jpeg_data})
        bottleneck = run_bottleneck_on_image(sess, distorted_image_data,
                                             resized_input_tensor,
                                             bottleneck_tensor)
    ground_truth = np.zeros(class_count, dtype=np.float32)
    ground_truth[label_index] = 1.0
    bottlenecks.append(bottleneck)
    ground_truths.append(ground_truth)
    
    return bottlenecks, ground_truths              

In [30]:
def should_distort_images(flip_left_right,random_crop,random_scale,random_brightness):
    
    """input flags로 부터 어떠한 왜곡이 가능하다면 사용
    인자들(Args):
    flip_left_right: boolean  이미지를 수평으로 랜덤하게 오른쪽 왼쪽으로 바꿀수 있는지
    random_crop: 크롭박스에 사용되는 전체 마진을 세팅해주는 int percentage
    random_brightness: 픽셀값을 곱해줄수있는 int range
    반환값들(Returns):
    왜곡이 적용되야하는지 아닌지 알려주는 boolean 값
    """
    
    return (flip_left_right or (random_crop !=0) or (random_scale !=0) or (random_brightness !=0))


def add_input_distortions(flip_left_right,random_crop,random_scale,random_brightness):
    """구체적인 몇몇 왜곡을 적용하여 이미지를 생성함"""
    
    
    jpeg_data=tf.placeholder(tf.string,name="DistortJPGInput")
    decoded_image=tf.image.decode_jpeg(jpeg_data,channels=MODEL_INPUT_DEPTH)
    decoded_image_as_float=tf.cast(decoded_image,dtype=tf.float32)
    decoded_image_4d=tf.expand_dims(decoded_image_as_float,0)
    margin_scale=1.0+(random_crop/100.0)
    resize_scale=1.0+(random_crop/100.0)
    
    margin_scale_value=tf.constant(margin_scale)
    resize_scale_value=tf.random_uniform(tensor_shape.scalar(),minval=1.0,maxval=resize_scale)
    
    scale_value=tf.multiply(margin_scale_value,resize_scale_value)
    
    precrop_width=tf.multiply(scale_value,MODEL_INPUT_WIDTH)
    precrop_height=tf.multiply(scale_value,MODEL_INPUT_HEIGHT)
    precrop_shape=tf.stack([precrop_height,precrop_width])
    precrop_shape_as_int=tf.cast(precrop_shape,dtype=tf.int32)
    precropped_image=tf.image.resize_bilinear(decode_image_4d,precrop_shape_as_int)
    precropped_image_3d=tf.squeeze(precropped_image,squeeze_dims=[0])
    cropped_image=tf.random_crop(precropped_image_3d,[MODEL_INPUT_HEIGHT,MODEL_INPUT_WIDTH,MODEL_INPUT_DEPTH])
    
    if flip_left_right:
        flipped_image=tf.image.random_flip_left_right(cropped_image)
    else:
        flipped_image=cropped_image
    
    brightenss_min=1.0-(random_brightness/100.0)
    brightenss_max=1.0+(random_brightness/100.0)
    brightness_value=tf.random_uniform(tensor_shape.scalar(),
                                      minval=brightenss_min,maxval=brightenss_max)
    brightened_image=tf.multiply(flipped_image,brightness_value)
    distort_result=tf.expand_dims(brightened_image,0,name="DistortResult")
    
    return jpeg_data,distort_result



In [31]:
def variable_summaries(var):
    """중간에 계측해서 보고싶은 값들 출력, tensorboard visualization"""
    with tf.name_scope("summaries"):
        mean=tf.reduce_mean(var)
        tf.summary.scalar('mean',mean)
        with tf.name_scope('stddev'):
            stddev=tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar("stddev",stddev)
        tf.summary.scalar("max",tf.reduce_max(var))
        tf.summary.scalar("min",tf.reduce_min(var))
        tf.summary.histogram('histogram',var)
        
        
def add_final_training_ops(class_count,final_tensor_name,bottleneck_tensor):
    """트레이닝에 소프트 맥스를 더하고 fully_connected layer를 더한다."""
    with tf.name_scope("input"):
        bottleneck_input=tf.placeholder_with_default(bottleneck_tensor,shape=[None,BOTTLENECK_TENSOR_SIZE],
                                                    name='BottleneckInputPlaceholder')
        ground_truth_input=tf.placeholder(tf.float32,[None,class_count],
                                         name="GroungTruthInput")
        
    layer_name='final_training_ops'
    with tf.name_scope(layer_name):
        with tf.name_scope('weights'):
            initial_value=tf.truncated_normal([BOTTLENECK_TENSOR_SIZE,class_count],stddev=0.001)
            layer_weights=tf.Variable(initial_value,name="final_weights")
            
            variable_summaries(layer_weights)
        with tf.name_scope("biases"):
            layer_biases=tf.Variable(tf.zeros([class_count]),name="final_biases")
            variable_summaries(layer_biases)
        with tf.name_scope("wx_plus_b"):
            logits=tf.matmul(bottleneck_input,layer_weights)+layer_biases
            tf.summary.histogram("pre_activation",logits)
            
    final_tensor=tf.nn.softmax(logits,name=final_tensor_name)
    tf.summary.histogram("activation",final_tensor)
    
    with tf.name_scope('cross_entropy'):
        cross_entropy=tf.nn.softmax_cross_entropy_with_logits(labels=ground_truth_input,logits=logits)
        with tf.name_scope('total'):
            cross_entropy_mean=tf.reduce_mean(cross_entropy)
    tf.summary.scalar("cross_entropy",cross_entropy_mean)
    
    with tf.name_scope("train"):
        optimizer=tf.train.GradientDescentOptimizer(FLAGS.learning_rate)
        train_step=optimizer.minimize(cross_entropy_mean)
        
    return (train_step, cross_entropy_mean, bottleneck_input, ground_truth_input,
          final_tensor)

In [32]:
def add_evaluation_step(result_tensor,ground_truth_tensor):
    """Inserts the operations we need to evaluate the accuracy of our results.
  Args:
    result_tensor: The new final node that produces results.
    ground_truth_tensor: The node we feed ground truth data
    into.
  Returns:
    Tuple of (evaluation step, prediction).
  """
    with tf.name_scope('accuracy'):
        with tf.name_scope("correct_prediction"):
            prediction=tf.argmax(result_tensor,1)
            correct_prediction=tf.equal(prediction,tf.argmax(ground_truth_tensor,1))
        with tf.name_scope("accuracy"):
            evaluation_step=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    tf.summary.scalar('accuracy',evaluation_step)
    
    return evaluation_step,prediction

In [37]:
def main():
    #텐서보드의 summarie를 write할 directory를 설정한다.
    if tf.gfile.Exists(FLAGS.summaries_dir):
        tf.gfile.DeleteRecursively(FLAGS.summaries_dir)
    tf.gfile.MakeDirs(FLAGS.summaries_dir)
    
    #pre_train 된 graph를생성한다.
    maybe_download_and_extract()
    graph,bottleneck_tensor,jpeg_data_tensor,resized_image_tensor=(create_inception_graph())
    
    # 폴더 구조를 살펴보고, 모든 이미지에 대한 lists를 생성한다.
    image_lists=create_image_lists(FLAGS.image_dir,FLAGS.testing_percentage,FLAGS.validation_percentage)
    class_count=len(image_lists.keys())
    
    if class_count == 0:
        print('No valid folders of images found at ' + FLAGS.image_dir)
        return -1
    if class_count == 1:
        print('Only one valid folder of images found at ' + FLAGS.image_dir +
              ' - multiple classes are needed for classification.')
        return -1
    
    #커맨드 라인 flag에 대한 distortion에 관련된 설정이 있으면 distortion을 적용한다.
    do_distort_images=should_distort_images(FLAGS.flip_left_right,FLAGS.random_crop,FLAGS.random_scale,
                                           FLAGS.random_brightness)
    
    with tf.Session(graph=grraph) as sess:
        if do_distort_imagges:
            (distorte_jpedg_data_tensor,
             distorted_image_tensor)=add_input_distortion(FLAGS.flip_left_right,FLAGS.random_crop,
                                                         FLAGS.bottleneck_dir,jpeg_data_tensor,
                                                         bottleneck_tensor)
        else:
            #우리가 계산된 bottlenck 이미지 summaries를 가지고있다.
            che_bottlenecks(sess,image_lists,FLAGS.image_dir,FLAGS.bottleneck_dir,jpeg_data_tensor,
                           bottleneck_tensor)
            
            #우리가 학습시킬 새로운 레이어를 추가한다.
        (train_step,cross_entropy,bottleneck_input,
         ground_truth_input,final_tensor)=add_final_training_ops(len(image_lists_keys()),
                                                                FLAGS.final_tensor_name,
                                                                bottleneck_tensor)
        
        # 우리의 새로운 layer의 정확도를 평가하기 위한 새로운 operation을 생성한다.
        evaluation_step,prediction=add_evaluation_step(final_tensor,ground_truth_input)
        
        #모든 summaries를 합치고 summaries_dir에 쓴다.
        merged_tf.summary.merage_all()
        train_writer=tf.summary.FileWriter(FLAGS.summaries_dir+'/train',sess.graph)
        
        #모든 가중치들과 그들의 초기값들을 설정한다.
        init=tf.global_variables_initializer()
        sess.run(init)
        
        #커맨드라인에서 지정한 횟수만큼 학습을 진행한다.
        for i in range(FLAGS.how_many_training_steps):
            #bottleneck 값들의 batch를 얻는다. 이는 매번 distortion을 적용하고, 계산하거나,
            # disk에 저장된 chache로부터 얻을 수 있다.
            
            if do_distort_images:
                (train_bottlenecks,
                 train_ground_truth) = get_random_distorted_bottlenecks(
                sess, image_lists, FLAGS.train_batch_size, 'training',
                FLAGS.image_dir, distorted_jpeg_data_tensor,
                distorted_image_tensor, resized_image_tensor, bottleneck_tensor)
            else:
                (train_bottlenecks,
                 train_ground_truth, _) = get_random_cached_bottlenecks(
                sess, image_lists, FLAGS.train_batch_size, 'training',
                FLAGS.bottleneck_dir, FLAGS.image_dir, jpeg_data_tensor,
                bottleneck_tensor)
            #graph에 bottleneck과 ground_truth를 feed하고, training_step을 진행한다. 
            #tnesor보드를 위한 merged op를 이용해서 trianing summaries를 캡쳐한다.
            
            train_summary,_=sess.run([merged,train_step],feed_dict={bottleneck_input:train_bottlenecks,
                                                                   ground_truth_input:train_ground_truth})
            train_writer.add_summary(train_summary,i)
            
            # 일정 step마다 graph의 training이 얼마나 잘되고 있는지 출력한다. 
            is_last_step=(i+1==FLAGS.how_many_training_steps)
            if (i%FLAGS.eval_step_interval)==0 or is_last_step:
                train_accuracy,cross_entropy_value=sese.run(
                [evaluation_step,cross_entropy],feed_dict={bottleneck_input:train_bottlenecks,
                                                          ground_truth_input:train_ground_truth})
                print("%s: step %d: train accuracy = %.1f%%" %(datetime.now(),i,train_accuracy*100))
                print("%s: step %d: cross entropy = %f" %(datetime.now(),i,cross_entropy))
                
                validation_bottlenecks,validation_ground_truth,_=(
                get_random_cached_bottlenecks(
                    sess,image_lists,FLAGS.validation_batch_size,'validation',
                    FLAGS.bottlenecks_dir,FLAGS.image_dir,jpeg_data_tensor,
                    bottleneck_tensor))
                # validation을 진행한다.
                #텐서보드을 위한 merged op 진행
                validation_summary,validation_accuracy=sess.run(
                [merged,evaluation_step],feed_dict={bottleneck_input:validation_bottlenecks,
                                                   ground_truth_input:validation_ground_truth})
                validation_writer.add_summary(validation_summary,i)
                print("%s: step %d: validation accuracy = %.1f%% (N=%d)" %(datetime.now(),i,validation_accuracy*100,
                                                                          len(validation_bottlenecks)))
                
            # 테스트 평가를 하자
            testbottlenecks,test_ground_truth,test_filenames=(
                   get_random_cached_bottlenecks(sess,image_lists,FLAGS.test_batch_size,'testing',FLAGS.bottleneck_dir,
                                                FLAGS.image_dir,jpeg_data_tensor,bottleneck_tensor))
            
            test_accuracy,predictions=sess.run(
                   [evaluation_step,prediction],
                    feed_dict={bottleneck_input:test_bottlenecks, ground_truth_input:test_ground_truth})
            
            print("final test accuracy = %.1f%% (N=%d)" %(test_accuracy*100,len(test_bottlenecks)))
            
            if FLAGS.print_misclassified_test_images:
                print("===MISCLASSIFIED TEST IMAGES===")
                for i, test_filename in enumerate(test_filenames):
                    if predictions[i] != test_ground_truth[i].argmax():
                        print("%70s %s"%(test_filename,list(image_lists.keys())[predictions[i]]))
                        
            # 학습된 그래프와 weight 들을 포함한 레이블을 쓴다.
            output_graph_def=graph_util.convert_variables_to_constant(
                sess,graph.as_graph_def(),[FLAGS.final_tensor_name])
            with gfile.FastGFile(FLASGS.output_graph,'wb') as f:
                f.write(output_graph_def.SerializeToString())
            with gfile.FastGFile(FLASG.output_labes,'w') as f:
                f.write("\n".join(image_lists.keys()) + '\n')
                


In [40]:
if __name__ == '__main__':
  parser = argparse.ArgumentParser()
  parser.add_argument(
      '--image_dir',
      type=str,
      default='',
      help='Path to folders of labeled images.'
  )
  parser.add_argument(
      '--output_graph',
      type=str,
      default='/tmp/output_graph.pb',
      help='Where to save the trained graph.'
  )
  parser.add_argument(
      '--output_labels',
      type=str,
      default='/tmp/output_labels.txt',
      help='Where to save the trained graph\'s labels.'
  )
  parser.add_argument(
      '--summaries_dir',
      type=str,
      default='/tmp/retrain_logs',
      help='Where to save summary logs for TensorBoard.'
  )
  parser.add_argument(
      '--how_many_training_steps',
      type=int,
      default=1000,
      help='How many training steps to run before ending.'
  )
  parser.add_argument(
      '--learning_rate',
      type=float,
      default=0.01,
      help='How large a learning rate to use when training.'
  )
  parser.add_argument(
      '--testing_percentage',
      type=int,
      default=10,
      help='What percentage of images to use as a test set.'
  )
  parser.add_argument(
      '--validation_percentage',
      type=int,
      default=10,
      help='What percentage of images to use as a validation set.'
  )
  parser.add_argument(
      '--eval_step_interval',
      type=int,
      default=10,
      help='How often to evaluate the training results.'
  )
  parser.add_argument(
      '--train_batch_size',
      type=int,
      default=100,
      help='How many images to train on at a time.'
  )
  parser.add_argument(
      '--test_batch_size',
      type=int,
      default=-1,
      help="""\
      How many images to test on. This test set is only used once, to evaluate
      the final accuracy of the model after training completes.
      A value of -1 causes the entire test set to be used, which leads to more
      stable results across runs.\
      """
  )
  parser.add_argument(
      '--validation_batch_size',
      type=int,
      default=100,
      help="""\
      How many images to use in an evaluation batch. This validation set is
      used much more often than the test set, and is an early indicator of how
      accurate the model is during training.
      A value of -1 causes the entire validation set to be used, which leads to
      more stable results across training iterations, but may be slower on large
      training sets.\
      """
  )
  parser.add_argument(
      '--print_misclassified_test_images',
      default=False,
      help="""\
      Whether to print out a list of all misclassified test images.\
      """,
      action='store_true'
  )
  parser.add_argument(
      '--model_dir',
      type=str,
      default='/tmp/imagenet',
      help="""\
      Path to classify_image_graph_def.pb,
      imagenet_synset_to_human_label_map.txt, and
      imagenet_2012_challenge_label_map_proto.pbtxt.\
      """
  )
  parser.add_argument(
      '--bottleneck_dir',
      type=str,
      default='/tmp/bottleneck',
      help='Path to cache bottleneck layer values as files.'
  )
  parser.add_argument(
      '--final_tensor_name',
      type=str,
      default='final_result',
      help="""\
      The name of the output classification layer in the retrained graph.\
      """
  )
  parser.add_argument(
      '--flip_left_right',
      default=False,
      help="""\
      Whether to randomly flip half of the training images horizontally.\
      """,
      action='store_true'
  )
  parser.add_argument(
      '--random_crop',
      type=int,
      default=0,
      help="""\
      A percentage determining how much of a margin to randomly crop off the
      training images.\
      """
  )
  parser.add_argument(
      '--random_scale',
      type=int,
      default=0,
      help="""\
      A percentage determining how much to randomly scale up the size of the
      training images by.\
      """
  )
  parser.add_argument(
      '--random_brightness',
      type=int,
      default=0,
      help="""\
      A percentage determining how much to randomly multiply the training image
      input pixels up or down by.\
      """
  )
  FLAGS, unparsed = parser.parse_known_args()
#   tf.app.run(main=main, argv=[sys.argv[0]] + unparsed)
  tf.app.run(main=main,argv=[sys.argv[0]]+unparsed)

TypeError: main() takes 0 positional arguments but 1 was given